In [41]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

In [2]:
np.set_printoptions(precision=6)

In [3]:
data = datasets.load_diabetes()

In [4]:
feature_names = ['age', 'sex', 'bmi', 'bpi', 's1', 's2', 's3', 's4', 's5', 's6']

In [5]:
trn, test, y_trn, y_test = train_test_split(data.data, data.target, test_size=0.2)

In [6]:
trn.shape, test.shape

((353, 10), (89, 10))

In [7]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

## linalg.lstqr

In [8]:
trn_int = np.c_[trn, np.ones(trn.shape[0])]
test_int = np.c_[test, np.ones(test.shape[0])]

In [9]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelsd")

500 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelsy")

639 µs ± 173 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelss")

419 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Naive Solution

In [12]:
def ls_naive(A, b):
    return np.linalg.inv(A.T @ A) @ A.T @ b

In [13]:
%timeit coeffs_naive = ls_naive(trn_int, y_trn)

141 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
coeffs_naive = ls_naive(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_naive)

(53.548912708699355, 43.972565025497275)

## Normal Equations(Cholesky)

In [15]:
A = trn_int

In [16]:
b = y_trn

In [17]:
AtA = A.T @ A
Atb = A.T @ b

In [18]:
R = scipy.linalg.cholesky(AtA)

In [19]:
np.set_printoptions(suppress=True, precision=4)
R

array([[ 0.9131,  0.1374,  0.1998,  0.3236,  0.2384,  0.2005, -0.084 ,
         0.1908,  0.266 ,  0.2868,  0.2829],
       [ 0.    ,  0.8823,  0.0448,  0.169 , -0.0167,  0.0889, -0.3356,
         0.2629,  0.0969,  0.1256, -0.2946],
       [ 0.    ,  0.    ,  0.8927,  0.3083,  0.192 ,  0.2016, -0.3123,
         0.3555,  0.365 ,  0.2776,  0.0619],
       [ 0.    ,  0.    ,  0.    ,  0.7853,  0.0845,  0.0116,  0.0411,
        -0.0038,  0.1472,  0.1502,  0.7479],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.8234,  0.756 ,  0.1286,
         0.382 ,  0.3027,  0.1392, -0.8116],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.3591, -0.3818,
         0.2693, -0.3048, -0.0312,  0.8081],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.6338,
        -0.5112, -0.5295, -0.1511,  0.4173],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.2938, -0.0339,  0.051 ,  0.1914],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
 

In [20]:
np.linalg.norm(AtA - R.T @ R)

1.136873527164815e-13

In [21]:
w = scipy.linalg.solve_triangular(R, Atb, lower=False, trans='T')

In [22]:
np.linalg.norm(R.T @ w - Atb)

1.4551915228366852e-11

In [23]:
coeffs_chol = scipy.linalg.solve_triangular(R, w, lower=False)

In [24]:
np.linalg.norm(R @ coeffs_chol - w)

2.864289338439558e-14

In [25]:
def ls_chol(A, b):
    R = scipy.linalg.cholesky(A.T @ A)
    w = scipy.linalg.solve_triangular(R, A.T @ b, trans='T')
    return scipy.linalg.solve_triangular(R, w)

In [26]:
%timeit coeffs_chol = ls_chol(trn_int, y_trn)

483 µs ± 207 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
coeffs_chol = ls_chol(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_chol)

(53.548912708699525, 43.97256502549738)

## QR Factorization

In [28]:
def ls_qr(A, b):
    Q, R = scipy.linalg.qr(A, mode='economic')
    return scipy.linalg.solve_triangular(R, Q.T @ b)

In [29]:
%timeit coeffs_qr = ls_qr(trn_int, y_trn)

545 µs ± 232 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
coeffs_qr = ls_qr(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_qr)

(53.54891270869954, 43.97256502549741)

## SVD

In [31]:
def ls_svd(A, b):
    m, n = A.shape
    U, sigma, Vh = scipy.linalg.svd(A, full_matrices=False, lapack_driver='gesdd')
    w = (U.T @ b)/sigma
    return Vh.T @ w

In [32]:
%timeit coeffs_svd = ls_svd(trn_int, y_trn)

598 µs ± 234 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
coeffs_svd = ls_svd(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_svd)

(53.54891270869952, 43.97256502549742)

## Timing Comparison

In [34]:
import timeit
import pandas as pd

In [35]:
def scipylstq(A, b):
    return scipy.linalg.lstsq(A, b)[0]

In [36]:
row_names = ['Normal Eqns- Naive',
            'Normal Eqns- Cholesky',
            'QR Factorization',
            'SVD',
            'Scipy lstsq']
name2func = {'Normal Eqns- Naive': 'ls_naive',
            'Normal Eqns- Cholesky': 'ls_chol',
            'SVD': 'ls_svd',
            'Scipy lstsq': 'scipylstq'}

In [37]:
m_array = np.array([100, 1000, 10000])
n_array = np.array([20, 100, 1000])

In [38]:
index = pd.MultiIndex.from_product([m_array, n_array], names=['# rows', '# cols'])

In [39]:
pd.options.display.float_format = '{:,.6f}'.format
df = pd.DataFrame(index=row_names, columns=index)
df_error = pd.DataFrame(index=row_names, columns=index)

In [ ]:
# %%prun
for m in m_array:
    for n in n_array:
        if m >= n:        
            x = np.random.uniform(-10,10,n)
            A = np.random.uniform(-40,40,[m,n])   # removed np.asfortranarray
            b = np.matmul(A, x) + np.random.normal(0,2,m)
            for name in row_names:
                fcn = name2func[name]
                t = timeit.timeit(fcn + '(A,b)', number=5, globals=globals())
                df.set_value(name, (m,n), t)
                coeffs = locals()[fcn](A, b)
                reg_met = regr_metrics(b, A @ coeffs)
                df_error.set_value(name, (m,n), reg_met[0])

In [ ]:
df

In [ ]:
df_error

In [ ]:
store = pd.HDFStore('least_squares_results.h5')

In [ ]:
store['df'] = df

## A case where QR is the best

In [51]:
m=100
n=15
t=np.linspace(0, 1, m)

In [44]:
# Vandermonde matrix
A=np.stack([t**i for i in range(n)], 1)

In [45]:
b=np.exp(np.sin(4*t))

# This will turn out to normalize the solution to be 1
b /= 2006.787453080206

In [48]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(t, b)